In [1]:
from dataConnection import DataConnector
from glob import glob
files = glob("../data/dataset/supply_chain/*.xls")

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

from datetime import datetime

from mlxtend.frequent_patterns import association_rules, apriori, fpgrowth
from mlxtend.preprocessing import TransactionEncoder

import sklearn
from sklearn.decomposition import TruncatedSVD

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus']= False 

import platform
if platform.system() == 'Darwin': 
    plt.style.use('seaborn-darkgrid') 
    rc('font', family = 'AppleGothic') 

In [3]:
connection=DataConnector(files)
df=connection.df_preprocess()

Connecting
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (6693331) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size

In [4]:
df

,day,customer,product,quantity,aggregate,ym
0,2020-02-01,군산봉정요양병원**,1042594,3.0,3150.0,202002
1,2020-02-01,군산봉정요양병원**,1028277,2.0,3900.0,202002
2,2020-02-01,군산봉정요양병원**,1004327,110.0,10890.0,202002
3,2020-02-01,군산봉정요양병원**,1049419,1.0,14839.0,202002
4,2020-02-01,군산봉정요양병원**,1038658,1.0,3355.0,202002
...,...,...,...,...,...,...
33536,2020-11-30,xx늘푸른요양병원,1028336,2.0,5880.0,202011
33537,2020-11-30,xx늘푸른요양병원,1050467,9.0,15840.0,202011
33538,2020-11-30,xx늘푸른요양병원,1087500,4.0,10160.0,202011
33540,2020-11-30,xx늘푸른요양병원,1087488,1.0,14300.0,202011


# Dataset 1.
### Condition
#### - Latest 3months
#### - Over 75% of product quantile

In [31]:
df.customer.unique()

array(['군산봉정요양병원**', '군산솔한방병원', '군산정다운병원', '군산효자요양병원**', '군산효자요양병원[소모품]',
       '다생한방병원', '더세움병원', '덕진경찰서', '박천수정형외과의원', '사랑요양병원**', '삼성휴먼빌[1]**',
       '연수병원', '예수병원[본관]', '예수병원[재활]', '우석대부속한방병원', '전주수병원', '전주우리병원**',
       '전주학산요양병원', '전주행복한요양병원', '주랑한방병원', '해맑은요양병원', '해맑은요양병원[암병동]',
       '효사랑전주병원', '효사랑전주병원[농산]', 'A-푸드마스터★', 'ABC[배송팀]', 'x고려병원',
       'x만인요양병원', 'x전주굿모닝요양원', 'x필밸런스의원', 'xx늘푸른[5병동]', 'xx늘푸른요양병원',
       'xx토리앤쿡1호본점임시', '공동체협동[쉐마유치원]', '공동체협동조합', '군산효자요양병원[직원]',
       '노송천노인복지관[부식]', '농업기술실용화재단**', '누리푸드[동전주우체국]', '누리푸드[전주농어촌공사]',
       '누리푸드[전주환경청]', '누리푸드[정읍세움]', '덕진노인복지관', '문실버요양병원', '신성양로원',
       '안골노인복지관[부식]', '온누리푸드넷', '전라북도노인복지관', '전북농업기술원', '전주외과의원',
       '정성산부인과', '제이비식품', '즐거운집노인요양원', '즐거운집노인요양원[소모품]', '참사랑낙원**',
       '토리밥상[식당]', '평화사회복지관', '한국가스공사', '한국가스공사[소모품]', '한국전기안전공사',
       '한국전기안전공사[소모품]', '한국전기안전공사[직원]', 'X나눔리버티', '가은요양원',
       '군산봉정요양병원[소모품]', '군산정다운병원[소모품]', '누리3호점[군산도시가스]', '누리3호점[전주노동청]',
       '누리푸드[김제새만금]', '누리푸드[김제신흥]', '누

In [5]:
gunsan=df[df['customer']=='군산봉정요양병원**']
gunsan=gunsan[['customer','product','ym']]
gunsan

,customer,product,ym
0,군산봉정요양병원**,1042594,202002
1,군산봉정요양병원**,1028277,202002
2,군산봉정요양병원**,1004327,202002
3,군산봉정요양병원**,1049419,202002
4,군산봉정요양병원**,1038658,202002
...,...,...,...
32188,군산봉정요양병원**,1049107,202011
32189,군산봉정요양병원**,1000232,202011
32190,군산봉정요양병원**,1125630,202011
32191,군산봉정요양병원**,1120305,202011


In [38]:
grouped_ym=gunsan.groupby('ym').agg(list)
max(grouped_ym.index)
# len(grouped_ym['product'][-1])
# grouped_ym.to_frame().reset_index()

'202112'

In [7]:
lst_length=[]
for i in range(len(grouped_ym['product'])):
    length=len(set(grouped_ym['product'][i]))
    lst_length.append(length)
print(max(lst_length), min(lst_length))


364 285


In [8]:
months=str(int(max(df["ym"]))-int(2))
gunsan_3months = gunsan[gunsan["ym"] >= months]

In [9]:
# Calculate quantile 0.75%
quantile_75=(gunsan_3months['product'].value_counts()).quantile(q=0.75)
quantile_75

5.0

In [10]:
gunsan_proprecessed=gunsan_3months[gunsan_3months['product'].map(gunsan_3months['product'].value_counts())>quantile_75]
gunsan_proprecessed

,customer,product,ym
105,군산봉정요양병원**,1027181,202110
106,군산봉정요양병원**,1121019,202110
107,군산봉정요양병원**,1042594,202110
108,군산봉정요양병원**,1109397,202110
109,군산봉정요양병원**,1004291,202110
...,...,...,...
46227,군산봉정요양병원**,1121714,202112
46229,군산봉정요양병원**,1047170,202112
46230,군산봉정요양병원**,1086643,202112
46231,군산봉정요양병원**,1121115,202112


In [11]:
gunsan_product=gunsan_proprecessed.groupby('ym').agg(list)
gunsan_product

,customer,product
ym,,
202110,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1027181, 1121019, 1042594, 1109397, 1004291, ..."
202111,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1045177, 1040955, 1028140, 1109397, 1042594, ..."
202112,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1004291, 1121019, 1042594, 1109397, 1004291, ..."


In [12]:
dataset=[]
for c,p in gunsan_product.iterrows():
    dataset.append(p['product'])

In [13]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
gunsan_dataset = pd.DataFrame(te_ary, columns=te.columns_)
gunsan_dataset

,1001574,1002205,1002213,1002291,1003327,1004291,1005406,1019595,1019909,1021177,...,1140506,1140516,1141891,1141894,1142526,1142536,1142549,1142551,1142592,1142598
0,True,True,True,True,True,True,True,True,True,True,...,False,True,True,True,True,True,True,True,True,True
1,True,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,False,True,True,True,True,True,True,True,True


In [14]:
%%time
apriori(gunsan_dataset, min_support=0.1, use_colnames=True)

KeyboardInterrupt: 

In [15]:
%%time
fpgrowth(gunsan_dataset, min_support=0.1, use_colnames=True)

KeyboardInterrupt: 

In [16]:
%%time
frequent_itemsets = apriori(gunsan_dataset, min_support = 0.1, max_len = None, use_colnames=True)
rules = association_rules(frequent_itemsets)
rules.head()

KeyboardInterrupt: 

In [17]:
%%time
rules_gunsan=fpgrowth(gunsan_dataset, min_support=0.1, max_len=None, use_colnames=True)
rules_gunsan

KeyboardInterrupt: 

# Dataset 2.
### Condition
#### - Latest 6months
#### - Over 75% of product quantile

In [18]:
months=str(int(max(df["ym"]))-int(5))
gunsan_6months = gunsan[gunsan["ym"] >= months]


In [19]:
# Calculate quantile 0.75%
quantile_6months=(gunsan_6months['product'].value_counts()).quantile(q=0.75)
quantile_6months

7.0

In [20]:
gunsan_proprecessed_6months=gunsan_6months[gunsan_6months['product'].map(gunsan_6months['product'].value_counts())>quantile_75]
gunsan_proprecessed_6months

,customer,product,ym
105,군산봉정요양병원**,1027181,202110
106,군산봉정요양병원**,1121019,202110
107,군산봉정요양병원**,1042594,202110
108,군산봉정요양병원**,1109397,202110
109,군산봉정요양병원**,1004291,202110
...,...,...,...
42498,군산봉정요양병원**,1042594,202108
42499,군산봉정요양병원**,1004291,202108
42500,군산봉정요양병원**,1028143,202108
42501,군산봉정요양병원**,1121714,202108


In [21]:
gunsan_product_6months=gunsan_proprecessed_6months.groupby('ym').agg(list)
gunsan_product_6months

,customer,product
ym,,
202107,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1042765, 1109397, 1042594, 1121019, 1004291, ..."
202108,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1042765, 1109397, 1042594, 1121019, 1004291, ..."
202109,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1087212, 1121019, 1042594, 1109397, 1004291, ..."
202110,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1027181, 1121019, 1042594, 1109397, 1004291, ..."
202111,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1045177, 1040955, 1028140, 1109397, 1042594, ..."
202112,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1004291, 1121019, 1042594, 1109397, 1004291, ..."


In [22]:
dataset_6months=[]
for c,p in gunsan_product_6months.iterrows():
    dataset_6months.append(p['product'])

In [23]:
te = TransactionEncoder()
te_ary_6months = te.fit(dataset_6months).transform(dataset_6months)
gunsan_dataset_6months = pd.DataFrame(te_ary_6months, columns=te.columns_)
gunsan_dataset_6months

,1000112,1000284,1000419,1001402,1001574,1001602,1002198,1002205,1002213,1002291,...,1142526,1142536,1142549,1142551,1142592,1142598,1142732,1142859,1143798,1153790
0,False,True,True,True,False,False,True,True,True,True,...,True,True,True,True,True,True,True,True,False,False
1,True,True,True,True,True,False,True,True,True,False,...,True,True,True,True,True,False,False,True,False,False
2,True,True,True,True,True,True,False,False,False,False,...,True,True,True,True,True,True,False,True,True,True
3,True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
4,False,True,True,False,True,True,True,False,True,True,...,True,True,True,True,True,True,False,False,True,True
5,True,True,False,False,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,False


In [24]:
%%time
apriori(gunsan_dataset_6months, min_support=0.1, use_colnames=True)

KeyboardInterrupt: 

In [25]:
%%time
fpgrowth(gunsan_dataset_6months, min_support=0.1, use_colnames=True)

KeyboardInterrupt: 

In [26]:
%%time
frequent_itemsets_6months = apriori(gunsan_dataset_6months, min_support = 0.1, max_len = None, use_colnames=True)
rules_6months = association_rules(frequent_itemsets_6months)
rules_6months.head()

KeyboardInterrupt: 

In [27]:
%%time
rules_gunsan_6months=fpgrowth(gunsan_dataset_6months, min_support=0.1, max_len=None, use_colnames=True)
rules_gunsan_6months

KeyboardInterrupt: 